In [1]:
%run ../utils/commonImports.py
%run ../utils/tradingImports.py
%matplotlib inline

# Load Data

In [2]:
def calculate_EMA(history, col, time_period):
    return calculateTalib('EMA', {col: history[col].values}, {'timeperiod': time_period})


def add_EMA(df):
    df['blue'] = calculate_EMA(df, 'close', 8)
    df['green'] = calculate_EMA(df, 'close', 13)
    df['yellow'] = calculate_EMA(df, 'close', 21)
    df['red'] = calculate_EMA(df, 'close', 55)
    return df

In [3]:
def load_and_divide(db, table, from_date=None, to_date=None, train_ratio=0.5):
    data = load_trading_data(db, table, from_date, to_date)
    data = add_EMA(data)
    data_train, data_test = divide_train_and_test(data, train_ratio)
    return data, data_train, data_test


db = 'D:\\Dropbox\\My work\\krypl-project\\sqlite\\ploniex-chart-data\\USDT_BTC.db'
data, data_train, data_test = load_and_divide(db, 'chart_data', from_date='2017-01-01')
ohlc, ohlc_train, ohlc_test = data.copy(), data_train.copy(), data_test.copy()


features = ['red', 'yellow', 'green', 'blue']
data_manager_train = CurrencyDataManager(data_train['close'], data_train[features])
data_manager_test = CurrencyDataManager(data_test['close'], data_test[features])
wallet = {'usdt': 1000}
contract_pair = ContractPair.new('usdt', 'btc')

# Explore

In [5]:
correct_pattern = ohlc_train.query("red < yellow < green < blue").copy()

In [6]:
correct_pattern.shape

(5904, 14)

In [ ]:
def ratio(df, c1, c2):
    return (df[c1] / df[c2]) - 1


def distplot(df, col, x_max):
    plt.figure(figsize=[12, 6])
    plt.title(col, fontproperties=title_font)
    sns.distplot(df[col])
    plt.xlim([0, x_max])

correct_pattern['ratioRY'] = ratio(correct_pattern, 'yellow', 'red')
correct_pattern['ratioYG'] = ratio(correct_pattern, 'green', 'yellow')
correct_pattern['ratioGB'] = ratio(correct_pattern, 'blue', 'green')

In [ ]:
distplot(correct_pattern, 'ratioRY', 0.015)
distplot(correct_pattern, 'ratioYG', 0.005)
distplot(correct_pattern, 'ratioGB', 0.005)

# Implementation

In [4]:
from modeling.strategy import Strategy

class HoldStrategy(Strategy):
    def __init__(self, exchange, data_manager, contract_pair, trade_size):
        super().__init__(exchange, data_manager, contract_pair, trade_size, 0., 0.)    
    
    def trade(self):
        while self.data_manager.has_tick():            
            history, price = self.data_manager.tick(1)
            
            if history.shape[0] == 0:
                self.buy(price)
            else:
                continue
        
        if self.opened:
            self.sell_all(price)

In [88]:
from modeling.strategy import Strategy
from itertools import permutations

ORDERS = list(permutations('BGYR', 4))


class CustomStrategy(Strategy):
    def __init__(self, exchange, data_manager, contract_pair, trade_size, willing_loss, target_profit, params):
        super().__init__(exchange, data_manager, contract_pair, trade_size, willing_loss, target_profit)
        self.history_len = 1
        self.params = params
    
    def has_correct_order(self, _type, red, yellow, green, blue):
        order = self.params['order_' + _type]
        d = {'R': red, 'Y': yellow, 'G': green, 'B': blue}
        l = [d[order[i]] for i in range(len(order))]
        for i in range(len(l)-1):
            if l[i] < l[i+1]:
                return False
        return True
        
    
    def should_buy(self, red, yellow, green, blue):
        if self.trade_size >= self.exchange.wallet[contract_pair['priceContract']] * 1.1:
            return False
        
        order = self.params['order_buy']
        if not self.has_correct_order('buy', red, yellow, green, blue):
            return False
    
        d = {'R': red, 'Y': yellow, 'G': green, 'B': blue}
        l = [d[order[i]] for i in range(len(order))]
        thresholds = [self.params[f'r{i}_buy'] for i in range(1, len(order))]
        
        def ratio(x, y):
            return (x / y) - 1
        
        for i in range(len(l)-1):
            r = ratio(l[i], l[i+1])
            if ratio(l[i], l[i+1]) < thresholds[i]:
                return False
        
        return True
    
    def should_sell(self, red, yellow, green, blue):
        return self.has_correct_order('sell', red, yellow, green, blue)
    
    
    def trade(self):
        after_sell = False
        while self.data_manager.has_tick():            
            history, price = self.data_manager.tick(self.history_len)
            
            if history.shape[0] == 0:
                continue
            
            red, yellow, green, blue = history[0]
            if np.isnan(red):
                continue
            
            if not self.opened and self.should_buy(red, yellow, green, blue):
                self.buy(price)
                price_bought = price
            elif self.opened and self.is_risky(price_bought, price):
                self.sell_all(price)
                after_sell = True
            elif self.opened and self.is_target_satisfied(price_bought, price):
                self.sell_all(price)
                after_sell = True
            elif self.opened and self.should_sell(red, yellow, green, blue):
                self.sell_all(price)
                after_sell = True
        
        if self.opened:
            self.sell_all(price)

In [89]:
def create_strategy(data_manager_p, param_array):
    params = {
        'order_buy': ORDERS[0],
        'r1_buy': param_array[0],
        'r2_buy': param_array[1],
        'r3_buy': param_array[2],
        'order_sell': ORDERS[param_array[3]],
    }
    
    data_manager = deepcopy(data_manager_p)
    exchange = BackTestExchange(data_manager, deepcopy(wallet), 0.0025)
    strategy = CustomStrategy(exchange, data_manager, contract_pair, 100, willing_loss=0.05, target_profit=1, params=params)
    return data_manager, exchange, strategy


def hold_stats(data_manager_p):
    data_manager = deepcopy(data_manager_p)
    exchange = BackTestExchange(data_manager, deepcopy(wallet), 0.0025)
    strategy = HoldStrategy(exchange, data_manager, contract_pair, 100)
    strategy.trade()
    return strategy.stats('usdt').report()

# Learning

In [90]:
def create_train_strategy(param_array):
    return create_strategy(data_manager_train, param_array)

In [91]:
hold_stats_train = hold_stats(data_manager_train)
hold_stats_train

,usdt
startAmount,"1,000.0000"
numberOfTrades,1.0000
totalProfit,282.2111
avgProfit,282.2111
winPercentage,100.0000
avgWinTrade,282.2111
avgLossTrade,0.0000
profitFactor,inf
maxDrawdown,0.0000


In [92]:
data_manager, exchange, strategy = create_train_strategy([0.0005, 0.0001, 0.00015, 4])
strategy.trade()
r = strategy.stats('usdt').report()
r

,usdt
startAmount,"1,000.0000"
numberOfTrades,68.0000
totalProfit,115.9605
avgProfit,1.7053
winPercentage,47.0588
avgWinTrade,8.8779
avgLossTrade,-4.6704
profitFactor,1.6897
maxDrawdown,3.3437


In [93]:
data_train.shape

(12476, 14)

In [95]:
from random import Random
from time import time
from math import cos
from math import pi
from inspyred import ec
from inspyred.ec import terminators

MIN_FLOAT = 1 / 10**5
MAX_FLOAT = 1-MIN_FLOAT

def random_args_part(random):
    l = [random.randint(0, len(ORDERS)-1)]
    l += [random.uniform(MIN_FLOAT, MAX_FLOAT) for _ in range(3)]
    return l


def generate_rastrigin(random, args):
    return [random.uniform(MIN_FLOAT, MAX_FLOAT) for _ in range(3)] + [random.randint(0, len(ORDERS)-1)]


def bound_args(candidate, args):
    for i in range(3):#[1, 2, 3, 5, 6, 7]:
        candidate[i] = max(min(candidate[i], MAX_FLOAT), MIN_FLOAT)
    
    candidate[3] = int(max(min(candidate[3], len(ORDERS)-1), 0))
    
    return candidate    

def evaluate_rastrigin(candidates, args):
    fitness = []
    for cs in candidates:
        fit = f(cs)
        fitness.append(fit)
    return fitness


from scipy.optimize import minimize, brute
import sys
from math import log10

def optim_val(stats, hold_stats):
    profit = stats.loc['totalProfit', 'usdt']
    trades = stats.loc['numberOfTrades', 'usdt']
    val = profit * trades
    return val

def f(x):
    data_manager, exchange, strategy = create_train_strategy(x)
    strategy.trade()
    r = strategy.stats('usdt').report()
    optimization_value = optim_val(r, hold_stats_train)
    sys.stdout.write(f"\r{x}, {optimization_value}")
    sys.stdout.flush()
    return optimization_value


rand = Random()
rand.seed(int(time()))
es = ec.ES(rand)
es.terminator = terminators.evaluation_termination
final_pop = es.evolve(generator=generate_rastrigin,
                      evaluator=evaluate_rastrigin,
                      pop_size=100,
                      maximize=True,
                      bounder=bound_args,
                      max_evaluations=2000,
                      mutation_rate=0.25)
# Sort and print the best individual, who will be at index 0.
final_pop.sort(reverse=True)
print(final_pop[0])

[0.10993920591046438, 0.8424021950440189, 1e-05, 16], 0.0 4], 0.0, 0.00.0558

ValueError: 

In [ ]:
final_pop[-1]

In [ ]:
data_manager, exchange, strategy = create_train_strategy(final_pop[-1].candidate)
strategy.trade()
r = strategy.stats('usdt').report()
print(optim_val(r, hold_stats_train))
r

# Look on transactions

In [ ]:
len(strategy.exchange.transactions)

In [ ]:
%matplotlib notebook
plot_transactions(ohlc_train, strategy.exchange.transactions[:])

for c in ['red', 'yellow', 'green', 'blue']:
    plt.plot(range(ohlc_train.shape[0]), ohlc_train[c], c=c)

# Evaluate

In [ ]:
def create_test_strategy(param_array):
    return create_strategy(data_manager_test, param_array)

In [ ]:
data_manager, exchange, strategy = create_test_strategy(best_params['x'])
strategy.trade()
strategy.stats('usdt').report()

In [ ]:
plot_transactions(ohlc_test, strategy.exchange.transactions[:])

for c in ['red', 'yellow', 'green', 'blue']:
    plt.plot(range(ohlc_test.shape[0]), ohlc_test[c], c=c)